<a href="https://colab.research.google.com/github/kimwoonggon/publicservant_AI/blob/master/1_(HuggingFace%2BTF2)%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%ED%8F%89%EA%B0%80_%EA%B8%8D%EB%B6%80%EC%A0%95_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# python 3.11.8
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

d:\programming\cloud-term-venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\programming\cloud-term-venv\Lib\site-packages\transformers\deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

d:\programming\cloud-term-venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\tokenizer.json
loading configuration file config.json from cache a

In [4]:
def convert_data(data_df):
    global tokenizer

    SEQ_LEN = 32 #SEQ_LEN : 버트에 들어갈 인풋의 길이

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')

        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros

        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

BATCH_SIZE = 20
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "document"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"


In [5]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
SEQ_LEN = 32
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])
# 버트 아웃풋의 텐서의 shape은 [batch_size, 문장의 길이, 768]임

loading configuration file config.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading weights file model.safet

In [6]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
opt = tfa.optimizers.RectifiedAdam(lr=1.0e-5, weight_decay=0.0025, warmup_proportion=0.05)

d:\programming\cloud-term-venv\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
d:\programming\cloud-term-venv\Lib\site-packages\tensorflow_addons\optimizers\rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [7]:
def create_sentiment_bert():
  # 버트 pretrained 모델 로드
  model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
  # 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
  # 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

  sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
  return sentiment_model

In [12]:
sentiment_model = create_sentiment_bert()
sentiment_model.load_weights('D:/programming/cloud-term-project/deeplearning-model/review_to_score.h5')

d:\programming\cloud-term-venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at C:\Users\user\.cache\huggingface\hub\models--bert-base-multilingual-cased\snapshots\3f076fdb1ab68d5b2880cb87a0886f315b8146f8\config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_h

In [13]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []

    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

test 데이터 예측하기

In [25]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')

    num_zeros = token.count(0)
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x, verbose=0)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()

    # print("만족도 수치화 ->", predict_value)
    return predict_value[0]

In [15]:
evaluation_predict("히레카츠를 너무 좋아하는데 여기 진짜 찐찐찐 마쉿서여")

1/1 [==============================] - 3s 3s/step
만족도 수치화 -> [0.8757981]


array([0.8757981], dtype=float32)

In [16]:
evaluation_predict("맛있습니다. 웨이팅을 잘해야하지만, 버크셔k랑 맥주랑 조합이 미쳤습니다. 안시켰으면 후회할뻔")

1/1 [==============================] - 0s 93ms/step
만족도 수치화 -> [0.7579161]


array([0.7579161], dtype=float32)

In [44]:
evaluation_predict("바삭바삭하고 맛나요 버크셔랑 로스도 같이 먹었는데 다 맛있었어요")

1/1 [==============================] - 0s 171ms/step
만족도 수치화 -> [0.6937898]


array([0.6937898], dtype=float32)

In [46]:
evaluation_predict("나오자마자 먹었음에도 불구하고 아래쪽이 축축하게 젖어있었음")

1/1 [==============================] - 0s 132ms/step
만족도 수치화 -> [0.14313643]


array([0.14313643], dtype=float32)

In [47]:
evaluation_predict("생각보다 너무 실망함..돈까스 엄청 기대했는데 튀김도 고기도 기대 이하였음")

1/1 [==============================] - 0s 116ms/step
만족도 수치화 -> [0.07659423]


array([0.07659423], dtype=float32)

In [48]:
evaluation_predict("불향난다는 버크셔시켰는데 저는 개인적으로 불향이 별로였고 (향이 풍성해서 느끼함을 배로 증가 시키는 효과) 무엇보다 위쪽은 바삭한데 아랫쪽 튀김은 눅눅해서 먹기 거북")

1/1 [==============================] - 0s 101ms/step
만족도 수치화 -> [0.15108642]


array([0.15108642], dtype=float32)

In [49]:
evaluation_predict("제가 갔을때만 별로였나, 추천받아 간 곳이었는데 맛있는지도 잘 모르겠고.")

1/1 [==============================] - 0s 99ms/step
만족도 수치화 -> [0.19183655]


array([0.19183655], dtype=float32)

In [49]:
import json
from collections import defaultdict

with open('D:\\programming\\cloud-term-project\\crawler\\data\\user_dict.json', 'r', encoding='utf-8') as f:
    user_data = json.load(f)
user_data = user_data['유저 정보']
userid_to_username = dict()
for user in user_data:
    userid_to_username[user['userId']] = user['userName']

with open('D:\\programming\\cloud-term-project\\crawler\data\\restaurant_review_dict.json', 'r', encoding='utf-8') as f:
    restaurants = json.load(f)

with open('D:\\programming\\cloud-term-project\\crawler\data\\user_reviews_dict.json', 'r', encoding='utf-8') as f:
    user_reviews = json.load(f)
user_new_reviews = dict()
user_restaurant_score = dict()
user_reviews = user_reviews['유저별 리뷰 정보']
j=0
for user_review in user_reviews:
    print(j)
    j += 1
    for username, reviews in user_review.items():
        user_new_reviews[username] = []
        user_restaurant_score[username] = {}
        for review in reviews:
            restaurant_name = review['restaurantName']
            base_score = review['reviewScore']
            new_score = base_score + evaluation_predict(review['description'])
            user_new_reviews[username].append((new_score, restaurant_name))
        user_new_reviews[username].sort()
        L = len(user_new_reviews[username])
        if L != 0:
            for i, user_new_review in enumerate(user_new_reviews[username]):
                user_restaurant_score[username][user_new_review[1]] = (i+1)/L



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [52]:
print(user_new_reviews['맛집탐방중'])

[(1.0690260082483292, '양자강'), (1.1402607262134552, '마루스터디카페'), (1.1510370820760727, '카페051 진해용원점'), (1.168426752090454, '돈부리파스타 타임스퀘어점'), (1.185379981994629, '오엔708'), (1.2448145747184753, '큰돈가 본점'), (1.3702205121517181, '우도돼지네땅콩만두 올레본점'), (2.080969899892807, '호랑이젤라떡'), (2.090602822601795, '하하호호 우도본점'), (2.1409707963466644, '이디야커피 인천연수롯데점'), (2.1476408541202545, '용원제일냉면'), (2.1540286391973495, '이스트우드 도림점'), (2.155965730547905, '치킨선생 신길점'), (2.2237553894519806, '여수딸기모찌 본점'), (2.239997461438179, '무목'), (2.4786033034324646, '갓버터도나스'), (2.5916698575019836, '틴틴'), (3.0923813059926033, '우도왕자이야기'), (3.0984716713428497, '아마렐로'), (3.108148545026779, '또보겠지 떡볶이집 강남헬로몽키점'), (3.156870573759079, '명품달인김밥'), (3.190822094678879, '남다른감자탕 하남미사점'), (3.1954264789819717, '쌤김밥'), (3.21513794362545, '희락돈까스'), (3.244947776198387, '별천지가든'), (3.7729446291923523, '장수갈매기 본점'), (4.074467346072197, '제면청년'), (4.121199697256088, '스터디카페 루체'), (4.16318741440773, '명문제과'), (4.202491611242294, '명동게장'), (4.20941399037838, '

In [51]:
for i, restaurant in enumerate(restaurants['식당 리뷰 정보']):
    for restaurant_name, reviews in restaurant.items():
        score_sum = 0
        count = 0
        for j, review in enumerate(reviews):
            userid = review['userId']
            username = userid_to_username[userid]
            
            try:
                normalized_score = user_restaurant_score[username][restaurant_name]
                score_sum += normalized_score
                count += 1
            except:
                pass
        if count != 0:
            print(restaurant_name, '의 점수 ', score_sum / count, sep='')


톤쇼우 부산대본점의 점수 0.5358738706632385
나오리쇼쿠 부산대점의 점수 0.5547579831078299
뉴숯불통닭 본점의 점수 0.4935760105650998
스시심의 점수 0.5434747351661476
진국명가의 점수 0.45655196297676015
델라고 부산대점의 점수 0.6201946437340494
코하루의 점수 0.4352437133841976
수림식당의 점수 0.5154670335847443
카페테제의 점수 0.45028896740189556
개성상인의 점수 0.45790278382956995
카츠안의 점수 0.474562126217239
포맨티코의 점수 0.5891563818613904
소다미김밥의 점수 0.712313325751987
김해원조뒷고기의 점수 0.444733454539669
우마이도 부산대점의 점수 0.37467533159780125
대박횟집의 점수 0.6402263039511247
커피어웨이크 부산대점의 점수 0.8679050567595459
설봉돼지국밥의 점수 0.7375332950877226
황금칼국수의 점수 0.6123362222924257
부산대우쭈쭈의 점수 0.5496452881339442
보노베리의 점수 0.4795012567978274
원조밀면의 점수 0.6385425685425685
촨촨향양꼬치의 점수 0.627906976744186
김해뒷고기 고가의 점수 0.5063777929517089
쿠카이야의 점수 0.4008084848783601
칠성돌곱창의 점수 0.544670891096412
스타벅스 부산대정문점의 점수 0.4258373205741627
홍보석 숯불구이의 점수 0.9333333333333333
이흥용과자점 부산대직영점의 점수 0.6944153696526502
마린파스타 부산대점의 점수 0.6845079086713296
돈까스겸의 점수 0.6540093838583192
진주비봉식당의 점수 0.34158470395874324
미각반점의 점수 0.4086151368760064
호호닭집

In [55]:
with open('D:\\programming\\cloud-term-project\\crawler\data\\user_reviews_dict_new.json', 'r', encoding='utf-8') as f:
    user_reviews_new = json.load(f)

for j, review in enumerate(user_reviews_new['유저별 리뷰 정보']):
    username = review['username']
    reviews = review['reviewList']
    for i, real_review in enumerate(reviews):
        restaurant_name = real_review['restaurantName']
        user_reviews_new['유저별 리뷰 정보'][j]['reviewList'][i]['normalizedScore'] = int(user_restaurant_score[username][restaurant_name]*1000)/1000
        print( reviews[i]['normalizedScore'], username, restaurant_name)

with open("D:\\programming\\cloud-term-project\\crawler\data\\user_reviews_dict_normalized.json", 'w', encoding='utf-8') as outfile:
    new_data = json.dumps(user_reviews_new, ensure_ascii=False, indent=4)
    outfile.write(new_data)

0.786 맛집탐방중 현덕사
0.466 맛집탐방중 톤쇼우 부산대본점
0.4 맛집탐방중 명동게장
0.186 맛집탐방중 여수딸기모찌 본점
0.213 맛집탐방중 갓버터도나스
0.68 맛집탐방중 뚜레쥬르 삼성역점
0.226 맛집탐방중 틴틴
0.813 맛집탐방중 고기싸롱 신길점
0.586 맛집탐방중 파파야리프 코엑스점
0.546 맛집탐방중 식객
0.346 맛집탐방중 장수갈매기 본점
0.48 맛집탐방중 황남빵
0.36 맛집탐방중 제면청년
0.946 맛집탐방중 베트남 키친 Bep Viet 본점
0.773 맛집탐방중 연돈볼카츠 신길점
0.266 맛집탐방중 또보겠지 떡볶이집 강남헬로몽키점
0.106 맛집탐방중 호랑이젤라떡
0.533 맛집탐방중 언니의로망
0.306 맛집탐방중 쌤김밥
0.973 맛집탐방중 몰운대
0.413 맛집탐방중 오사카
0.64 맛집탐방중 성진묵은지감자탕 연수직영점
0.653 맛집탐방중 일성관
0.013 맛집탐방중 양자강
0.146 맛집탐방중 용원제일냉면
0.826 맛집탐방중 본참치
0.96 맛집탐방중 제일렌트카 순천영업소
0.933 맛집탐방중 몰운대유원지
0.333 맛집탐방중 별천지가든
0.32 맛집탐방중 희락돈까스
0.386 맛집탐방중 명문제과
0.986 맛집탐방중 다올
0.72 맛집탐방중 코인싱어동전노래연습장
0.6 맛집탐방중 풍미통닭
0.88 맛집탐방중 관산식당
0.666 맛집탐방중 명랑핫도그 진해용원점
0.04 맛집탐방중 카페051 진해용원점
0.626 맛집탐방중 쌀로떡볶이 본점
0.453 맛집탐방중 시간과불의맛 어화 용원본점
0.84 맛집탐방중 용원가야밀면
0.28 맛집탐방중 명품달인김밥
0.8 맛집탐방중 호호춘천닭갈비 본점
0.76 맛집탐방중 대한민국 진짜배기
0.693 맛집탐방중 영진돼지국밥수육
0.52 맛집탐방중 원조호수삼계탕 본점
0.893 맛집탐방중 가온칼국수 송도본점
0.706 맛집탐방중 스시민 영등포점
0.053 맛집탐방중 돈부리파스타 타임스퀘어점
0.866 맛집탐방중 숲에서 스터디카페 도림사거리점
0.613 맛집탐방중 월령코지펜션
0